In [ ]:
import os
import sys
_path = os.path.abspath('../')
if _path not in sys.path:
    sys.path.insert(0, _path)

import matplotlib as mpl
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
plt.style.use('apw-notebook')
%matplotlib inline
import numpy as np

import astropy.units as u
from astropy.io import ascii
import astropy.coordinates as coord
import gala.coordinates as gc
import emcee
import corner

from lowlats.likelihood import LinearVelocityModel

In [ ]:
# structure = 'Mon/GASS'
structure = 'A13'
l0 = 150. # reference longitude in degrees

In [ ]:
data = ascii.read('../data/all_mg_rr.ecsv', format='ecsv')
data = data[(data['structure'] == structure) & np.isfinite(data['v_gsr']) & np.isfinite(data['v_err'])]
# data = data[( ((data['structure'] == 'Mon/GASS') | (data['structure'] == 'A13')) & 
#              np.isfinite(data['v_gsr']) & np.isfinite(data['v_err']))]

In [ ]:
mg = data[data['tracer'] == 'MG']
rr = data[data['tracer'] == 'RRL']
len(mg), len(rr)

In [ ]:
plt.scatter(mg['l'], mg['v_gsr'])
plt.scatter(rr['l'], rr['v_gsr'])

In [ ]:
model = LinearVelocityModel(rrlyrae=rr, mgiants=mg, halo_sigma_v=100.)

In [ ]:
plt.scatter(mg['l']-l0, mg['v_gsr'])
plt.scatter(rr['l']-l0, rr['v_gsr'])

l_grid = np.linspace(data['l'].min(), data['l'].max(), 128) - l0
dv_dl0 = -1.
v0 = 10
plt.plot(l_grid, dv_dl0*l_grid + v0, marker='None')

In [ ]:
p0 = model.pack_pars(dv_dl=dv_dl0, v0=v0, lnV=np.log(25**2), f_mg=0.9, f_rr=0.5)

## MCMC parameters

In [ ]:
n_walkers = 64

## Initial conditions for walkers

In [ ]:
p0s = emcee.utils.sample_ball(p0, std=np.full_like(p0, 1E-4), size=n_walkers)

In [ ]:
sampler = emcee.EnsembleSampler(n_walkers, len(p0), lnpostfn=model)

In [ ]:
pos,prob,state,_ = sampler.run_mcmc(p0s, N=128)
sampler.reset()
pos[pos[:,1] < 200,1] = 250
pos[pos[:,2] < 6,2] = 6
pos[pos[:,3] < 0.5,3] = 0.9
pos,prob,state,_ = sampler.run_mcmc(pos, N=256)
sampler.reset()
pos,prob,state,_ = sampler.run_mcmc(pos, N=1024)

In [ ]:
sampler.chain.shape

In [ ]:
for i in range(sampler.chain.shape[0]):
    plt.plot(sampler.chain[i,:,4], marker='None', drawstyle='steps', color='k', alpha=0.1)
    
for i in range(sampler.chain.shape[0]):
    plt.plot(sampler.chain[i,:,3], marker='None', drawstyle='steps', color='r', alpha=0.1)
    
plt.ylim(0, 1)

In [ ]:
# flatchain = np.vstack(sampler.chain[:,1024::16])
flatchain = np.vstack(sampler.chain[:,::4])
flatchain[:,2] = np.sqrt(np.exp(flatchain[:,2]))

In [ ]:
labels = [r'${{\rm d}}v/{{\rm d}}l$ [{0} ${{\rm deg}}^{{-1}}$]'.format((u.km/u.s).to_string('latex_inline')), 
          r'$v_0$ [{0}]'.format((u.km/u.s).to_string('latex_inline')), 
          r'$\sigma_v$ [{0}]'.format((u.km/u.s).to_string('latex_inline')), 
          r'$f_{\rm MG}$', 
          r'$f_{\rm RR}$']
extents = [(-1.8,0.2), (50,300), (7,37), (0,1), (0,1)]

fig = corner.corner(flatchain, labels=labels, range=extents, plot_datapoints=False)
# fig.subplots_adjust(left=0.14, bottom=0.14, wspace=0.11, hspace=0.11)
# fig.savefig("/Users/adrian/projects/triand-rrlyrae/plots/posterior.pdf")

In [ ]:
print(np.mean(flatchain[:,0]), np.std(flatchain[:,0]))
print(np.mean(flatchain[:,1]), np.std(flatchain[:,1]))
print(np.mean(flatchain[:,2]), np.std(flatchain[:,2]))

### Numbers for paper

In [ ]:
map_dv_dl, map_v0, map_triand_sigma_v, map_f_mg, map_f_rr = sampler.flatchain[sampler.flatlnprobability.argmax()]

In [ ]:
print(map_f_mg, map_f_mg * len(mg))
print(map_f_rr) #, map_f_rrl * 141.)

In [ ]:
# number of M giants in longitude range of RR Lyrae
N_mg_raw = ((mg['l'] > rr['l'].min()) & (mg['l'] < rr['l'].max())).sum()

# N_rr_ps1 = 299 # GASS
N_rr_ps1 = 101 # A13
# N_rr_ps1 = 299+101
N_mg = map_f_mg * N_mg_raw
N_rr = map_f_rr * N_rr_ps1
N_rr / N_mg

In [ ]:
f_rrmg = (flatchain[:,4] * N_rr_ps1) / (flatchain[:,3] * N_mg_raw)

In [ ]:
fig,axes = plt.subplots(1,1,figsize=(6,6))

# frrl = sampler.flatchain[:,5]
axes.hist(f_rrmg, bins=np.linspace(0,1.,25), color='#666666', normed=True);
axes.axvline(0.5, lw=3.)

axes.plot([0.01,0.01], [0,4.5], lw=3., linestyle='--', marker=None, label='Disk')
axes.fill_betweenx(y=np.linspace(0,4.5,5), x1=0.42, x2=0.58, color='#bbbbbb')
# hack
p2 = mpl.patches.Rectangle((0, 0), 0., 0., color='#cccccc', label='Sgr/LMC')
axes.add_patch(p2)
axes.set_xlim(-0.05,1.05)
axes.set_ylim(0, 4.5)
axes.set_xlabel(r"$f_{\rm RR:MG}$")
axes.set_ylabel(r"pdf")
axes.legend(fontsize=16)
# axes.set_title('Mon/GASS')
axes.set_title('A13')

fig.savefig('../frrmg_a13.pdf')

### Membership probabilities

In [ ]:
sampler.chain.shape[1]

In [ ]:
norm = 0.0
mg_mem_prob = np.zeros(len(mg))
rr_mem_prob = np.zeros(len(rr))
for i in range(sampler.chain.shape[1]):
    for j in range(sampler.chain.shape[0]):
        ll_bg, ll_fg, ll_bg_rr, ll_fg_rr = sampler.blobs[i][j]
        mg_mem_prob += np.exp(ll_fg - np.logaddexp(ll_fg, ll_bg))
        rr_mem_prob += np.exp(ll_fg_rr - np.logaddexp(ll_fg_rr, ll_bg_rr))
        norm += 1
mg_mem_prob /= norm
rr_mem_prob /= norm

In [ ]:
(mg_mem_prob > 0.5).sum(), (rr_mem_prob > 0.5).sum()

### Combined figure

In [ ]:
mem_prob_thresh = 0.5
style = dict(marker='o', ms=5, linestyle='none', ecolor='#cccccc', elinewidth=2)
xlim = (260, 120)
ylim = (-300, 300)

In [ ]:
fig,axes = plt.subplots(2, 1, figsize=(10,9), sharex=True, sharey=True)

ax = axes[0]

members = mg_mem_prob > mem_prob_thresh
ax.errorbar(mg['l'][members], mg['v_gsr'][members], mg['v_err'][members], 
            color='k', label='$P>{0:.1f}$'.format(mem_prob_thresh), **style)
ax.errorbar(mg['l'][~members], mg['v_gsr'][~members], mg['v_err'][~members], 
            color='#777777', label='$P\leq {0:.1f}$'.format(mem_prob_thresh), 
            **style)

ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_ylabel(r"$v_{\rm GSR}\,[{\rm km} \, {\rm s}^{-1}]$")

ax.yaxis.set_ticks(np.arange(ylim[0], ylim[1]+1, 100))

ax.legend(loc='upper left', fontsize=16)
ax.text(255, -275, r'M giant', ha='left', va='bottom', fontsize=20)

# ------------------
# Bottom

ax = axes[1]

members = rr_mem_prob > mem_prob_thresh
ax.errorbar(rr['l'][members], rr['v_gsr'][members], rr['v_err'][members], 
             color='k', **style)
ax.errorbar(rr['l'][~members], rr['v_gsr'][~members], rr['v_err'][~members], 
             color='#777777', **style)

ax.set_xlabel(r"$l\,{\rm [deg]}$", fontsize=26)
ax.set_ylabel(r"$v_{\rm GSR}\,[{\rm km} \, {\rm s}^{-1}]$")

# lines
# ls = np.linspace(170, 90, 100)
# ax1.plot(ls, map_dv_dl*ls + map_v0, color='k', alpha=0.6, marker=None, linestyle='-', lw=2.)
# ax1.plot(ls, map_dv_dl*ls + map_v0 + map_triand_sigma_v, color='k', alpha=0.6, marker=None, linestyle='--', lw=2.)
# ax1.plot(ls, map_dv_dl*ls + map_v0 - map_triand_sigma_v, color='k', alpha=0.6, marker=None, linestyle='--', lw=2.)

ax.text(255, 275, r'RR Lyrae', ha='left', va='top', fontsize=20)

fig.subplots_adjust(wspace=0, hspace=0)
fig.savefig("../data.pdf")